In [13]:
import anndata
import pandas as pd
import glob
import pathlib

In [26]:
all_inte_files = glob.glob('Round2/*/mC-m3C-integration.csv')
mapped = [path.split('/')[1] for path in all_inte_files]

In [27]:
all_groups= [pathlib.Path(path).name for path in glob.glob('Round2/*')]
all_groups.remove('Snakefile')
all_groups.remove('notebooks')

In [42]:
all_inte_l2_df = []
for path in all_inte_files:
    group = path.split('/')[1]
    tmpdf = pd.read_csv(path, index_col = 0)
    all_inte_l2_df.append(tmpdf)
    tmpdf['mc_m3c-co_cluster_l2'] = group + '-' + tmpdf['leiden'].astype(str)

In [43]:
concat_df = pd.concat(all_inte_l2_df)

In [53]:
mc_meta = pd.read_csv('mC_META_230814.csv', index_col =0)
m3c_meta = pd.read_csv('m3C_META_230814.csv', index_col =0)

## annot m3C

In [149]:
mc_meta['mc_m3c-co_cluster_l2'] = concat_df['mc_m3c-co_cluster_l2']
m3c_meta['mc_m3c-co_cluster_l2'] = concat_df['mc_m3c-co_cluster_l2']

In [76]:
co_cluster_to_annot ={}
co_cluster_to_annot_ratio = {}
for co_cluster, tmpdf in mc_meta.groupby('mc_m3c-co_cluster_l2'):
    co_cluster_to_annot[co_cluster] = tmpdf['new_celltypes_0808'].value_counts().index[0]
    ratio = tmpdf['new_celltypes_0808'].value_counts().iloc[0] / tmpdf.shape[0]
    co_cluster_to_annot_ratio[co_cluster] = ratio

In [90]:
m3c_meta['CellType'] = m3c_meta['mc_m3c-co_cluster_l2'].map(co_cluster_to_annot)

In [106]:
un_mapped_clusters = m3c_meta[m3c_meta['CellType'].isna()]['mc_m3c-co_cluster_l1'].unique().tolist()

In [112]:
co_cluster_to_annot_l1 = {}
for _cluster in un_mapped_clusters:
    celltype = mc_meta[mc_meta['mc_m3c-co_cluster_l1'] == _cluster]['new_celltypes_0808'].value_counts().index[0]
    co_cluster_to_annot_l1[_cluster] =celltype 


In [126]:
m3c_meta.loc[m3c_meta['CellType'].isna(),'CellType'] = m3c_meta['mc_m3c-co_cluster_l1'].map(co_cluster_to_annot_l1).dropna()

In [ ]:
set(m3c_meta['CellType'].unique()) ^ set(mc_meta['new_celltypes_0808'].unique())

{'ABC NN',
 'LDT-PCG Gata3 Irx5 Gaba',
 'MOB-STR-CTX Inh IMN',
 'SNc-VTA-RAmb Foxa1 Dopa'}

In [133]:
m3c_meta.to_csv('m3C_META_230814.csv')

In [151]:
mc_meta['CellType'] = mc_meta['new_celltypes_0808']
mc_meta.to_csv('mC_META_230814.csv')